In [1]:
# libs
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBClassifier
import csv
import json

In [2]:
iris_data = pd.read_csv(
        "data/iris.data",
        header=None,
        names=["Sepal Length", "Sepal Width","Petal Length", "Petal Width", "Species"])

In [3]:
# Process Data
label_encoder = LabelEncoder()
iris_data["Species_Encoded"] = label_encoder.fit_transform(iris_data["Species"])

In [4]:
iris_data.head()

,Sepal Length,Sepal Width,Petal Length,Petal Width,Species,Species_Encoded
0,5.1,3.5,1.4,0.2,Iris-setosa,0
1,4.9,3.0,1.4,0.2,Iris-setosa,0
2,4.7,3.2,1.3,0.2,Iris-setosa,0
3,4.6,3.1,1.5,0.2,Iris-setosa,0
4,5.0,3.6,1.4,0.2,Iris-setosa,0


In [5]:
# Saved processed data to new file and json
iris_data.to_csv("data/encoded_data.csv")
options_title = iris_data["Species"].unique()
dict_encoder = {}
for item in options_title:
    dict_encoder[str(iris_data[iris_data["Species"] == item].iloc[0]["Species_Encoded"])] = item
with open("data/encoder.json", "w") as write_file:
        json.dump(dict_encoder, write_file, indent=4) 


In [6]:
# Make x and y data
y = iris_data["Species_Encoded"].copy()
x = iris_data.drop(["Species", "Species_Encoded"], axis=1)
x_train, x_valid, y_train, y_valid = train_test_split(x, y, train_size=0.75)

In [7]:
# Train model and save
model = XGBClassifier()
model.fit(x_train, y_train)
model.save_model("data/main_model.model")

C:\Users\Thiago Miranda\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [18:23:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [8]:
# Test model
predictions = model.predict(x_valid)
print(f"MAE: {str(mean_absolute_error(predictions, y_valid))}")

MAE: 0.05263157894736842
